# **IP1_Run1**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 8.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """ We are delighted to welcome you, our newest intern on the Analytics team of Massachusetts General Hospital! You have been placed in a challenging role where you will be tasked with solving a real-world problem in the field of medical physics. We are building a pilot program in Boston, and if successful, your work could be applied widely in hospitals with limited capacity in many countries.

You are responsible for determining the best treatment plan for 17 patients who require radiotherapy. Your goal is to optimize the use of two possible treatments: photon therapy and proton therapy. While proton therapy is known to target tumors more precisely, it is also more expensive and has limited capacity in many countries. Therefore, you will need to balance the benefits of proton therapy with its limitations and cost to create an effective treatment plan for each patient.

To determine the best course of action for each patient, you will use a scoring system called the Biological Equivalent Dose (BED). This system allows you to calculate the effectiveness of each patient’s treatment plan by considering the number of proton fractions that can be used while still achieving the highest possible BED.

We have n=17 patients who need radiotherapy. Each patient i needs 15 fractions, which can be photon fractions, proton fractions, or a mix of photon and proton fractions (e.g. 4 proton fractions and 11 photon fractions). We want to use the limited proton therapy capacity as best as possible. We can calculate the BED score for each patient when p proton fractions and 15-p photon fractions are used, as BEDi(p,15-p), i.e., the BED when p proton and 15-p photon fractions are delivered for patient i. The higher the score, the better.

The data file "ProblemData.csv" contains a 2D matrix of BED scores. It does not have an index. It was made in Excel and saved as csv. The columns are the number of proton fractions and each row represents a patient. In particular, the number at the (i,j) position is the score for patient i receiving j proton fractions.

Suppose that the total maximal capacity C is 100 proton fractions. To maximize the total BED scores for all the patients, which patients should get proton fractions, and how many should they get? Formulate an integer linear optimization model to solve this problem. Assume you know the value BEDi(j,15-j) for each patient i. """

In [5]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [6]:
print(response1.choices[0].message.content)

Parameters:

- C: Capacity of proton fractions = 100
- n: Total number of patients = 17
- BEDi(j, 15-j): Score for patient i receiving j proton fractions (for each i ∈ {1,2,3,...,17}, j ∈ {0,1,2,...15}). The score is given in our data.
- MaxProtonPerPatient: Maximum number of proton fractions that each patient can have =15

Decision Variables:
Let's denote Xi,j as a binary variable where i is the patient and j is the number of proton fractions. Xi,j is equal to 1 if patient i should get j proton fractions, and 0 if not.

Objective Function:

Maximize ∑_(i=1)^17 ∑_(j=0)^15 (BEDi(j, 15-j)*Xi,j)
We try to maximize the sum of all BED scores.


Constraints:

1. Total number of Proton Fractions: The total number of proton fractions delivered across all patients cannot exceed the total capacity C=100.
This can be formulated as ∑_(i=1)^17 ∑_(j=0)^15 (j*Xi,j) <= C

2. Single treatment per Patient: Each patient gets a single treatment plan i.e. a certain number of proton fractions between 0 and 

### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [8]:
Markdown(response2.choices[0].message.content)

The python Pyomo optimization code will look like below. The sample BEDi(j, 15-j) data is randomly generated between 0 and 100.

```python
from pyomo.environ import *

# create a model
model = ConcreteModel()

# index sets
model.Patients = RangeSet(1, 17)
model.Fractions = RangeSet(0, 15)

# parameters
model.C = Param(initialize=100)

#sample random BED data
import random
BED = {(i,j): random.randint(0,100) for i in model.Patients for j in model.Fractions}
model.BED = Param(model.Patients, model.Fractions, initialize=BED)

# decision variables
model.X = Var(model.Patients, model.Fractions, within=Binary)

# objective function
model.value = Objective(expr = sum(model.BED[i,j] * model.X[i,j] for i in model.Patients for j in model.Fractions), sense=maximize)

# constraints
def total_fractions_constraint(model):
    return sum(j * model.X[i,j] for i in model.Patients for j in model.Fractions) <= model.C
model.total_fractions = Constraint(rule=total_fractions_constraint)

def single_plan_constraint(model, i):
    return sum(model.X[i,j] for j in model.Fractions) == 1
model.single_plan = Constraint(model.Patients, rule=single_plan_constraint)

# Solve
solver = SolverFactory('glpk')
solver.solve(model)
``` 

In order to get the values of the variables after solving, add this print script:

```python
for i in model.Patients:
    for j in model.Fractions:
        print("Patient ", i, "Fractions ", j, ": ", model.X[i,j].value)
``` 

Note: For this specific problem, the type of mathematical programming problem has been established as an integer programming (IP) problem and since every variable can only have a value of either 0 or 1 this becomes a binary integer programming (BIP) problem.

### **Run the code Generated by GPT4**

In [59]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# index sets
model.Patients = RangeSet(1, 17)
model.Fractions = RangeSet(0, 15)

# parameters
model.C = Param(initialize=100)

# sample random BED data
import random
BED = {(i,j): random.randint(0,100) for i in model.Patients for j in model.Fractions}
model.BED = Param(model.Patients, model.Fractions, initialize=BED)

# decision variables
model.X = Var(model.Patients, model.Fractions, within=Binary)

# objective function
model.value = Objective(expr = sum(model.BED[i,j] * model.X[i,j] for i in model.Patients for j in model.Fractions), sense=maximize)

# constraints
def total_fractions_constraint(model):
    return sum(j * model.X[i,j] for i in model.Patients for j in model.Fractions) <= model.C
model.total_fractions = Constraint(rule=total_fractions_constraint)

def single_plan_constraint(model, i):
    return sum(model.X[i,j] for j in model.Fractions) == 1
model.single_plan = Constraint(model.Patients, rule=single_plan_constraint)

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

for i in model.Patients:
    for j in model.Fractions:
        print("Patient ", i, "Fractions ", j, ": ", model.X[i,j].value)
print(model.value())

Patient  1 Fractions  0 :  0.0
Patient  1 Fractions  1 :  0.0
Patient  1 Fractions  2 :  0.0
Patient  1 Fractions  3 :  1.0
Patient  1 Fractions  4 :  0.0
Patient  1 Fractions  5 :  0.0
Patient  1 Fractions  6 :  0.0
Patient  1 Fractions  7 :  0.0
Patient  1 Fractions  8 :  0.0
Patient  1 Fractions  9 :  0.0
Patient  1 Fractions  10 :  0.0
Patient  1 Fractions  11 :  0.0
Patient  1 Fractions  12 :  0.0
Patient  1 Fractions  13 :  0.0
Patient  1 Fractions  14 :  0.0
Patient  1 Fractions  15 :  0.0
Patient  2 Fractions  0 :  0.0
Patient  2 Fractions  1 :  0.0
Patient  2 Fractions  2 :  1.0
Patient  2 Fractions  3 :  0.0
Patient  2 Fractions  4 :  0.0
Patient  2 Fractions  5 :  0.0
Patient  2 Fractions  6 :  0.0
Patient  2 Fractions  7 :  0.0
Patient  2 Fractions  8 :  0.0
Patient  2 Fractions  9 :  0.0
Patient  2 Fractions  10 :  0.0
Patient  2 Fractions  11 :  0.0
Patient  2 Fractions  12 :  0.0
Patient  2 Fractions  13 :  0.0
Patient  2 Fractions  14 :  0.0
Patient  2 Fractions  15 :  

In [60]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# index sets
model.Patients = RangeSet(1, 17)
model.Fractions = RangeSet(0, 15)

# parameters
model.C = Param(initialize=100)

#START - inputting data - done by human
df = pd.read_csv("ProblemData (2).csv", header= None)

BED = {}
for i, row in df.iterrows():
    for j, dose in enumerate(row):
        BED[(i+1, j)] = dose
#END

# decision variables
model.X = Var(model.Patients, model.Fractions, within=Binary)

# objective function
model.value = Objective(expr = sum(BED[i,j] * model.X[i,j] for i in model.Patients for j in model.Fractions), sense=maximize)

# constraints
def total_fractions_constraint(model):
    return sum(j * model.X[i,j] for i in model.Patients for j in model.Fractions) <= model.C
model.total_fractions = Constraint(rule=total_fractions_constraint)

def single_plan_constraint(model, i):
    return sum(model.X[i,j] for j in model.Fractions) == 1
model.single_plan = Constraint(model.Patients, rule=single_plan_constraint)

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

for i in model.Patients:
    for j in model.Fractions:
      print("Patient ", i, "Fractions ", j, ": ", model.X[i,j].value)
print(model.value())

Patient  1 Fractions  0 :  0.0
Patient  1 Fractions  1 :  0.0
Patient  1 Fractions  2 :  0.0
Patient  1 Fractions  3 :  0.0
Patient  1 Fractions  4 :  0.0
Patient  1 Fractions  5 :  0.0
Patient  1 Fractions  6 :  0.0
Patient  1 Fractions  7 :  0.0
Patient  1 Fractions  8 :  0.0
Patient  1 Fractions  9 :  0.0
Patient  1 Fractions  10 :  0.0
Patient  1 Fractions  11 :  0.0
Patient  1 Fractions  12 :  0.0
Patient  1 Fractions  13 :  0.0
Patient  1 Fractions  14 :  0.0
Patient  1 Fractions  15 :  1.0
Patient  2 Fractions  0 :  0.0
Patient  2 Fractions  1 :  0.0
Patient  2 Fractions  2 :  0.0
Patient  2 Fractions  3 :  0.0
Patient  2 Fractions  4 :  0.0
Patient  2 Fractions  5 :  0.0
Patient  2 Fractions  6 :  0.0
Patient  2 Fractions  7 :  0.0
Patient  2 Fractions  8 :  1.0
Patient  2 Fractions  9 :  0.0
Patient  2 Fractions  10 :  0.0
Patient  2 Fractions  11 :  0.0
Patient  2 Fractions  12 :  0.0
Patient  2 Fractions  13 :  0.0
Patient  2 Fractions  14 :  0.0
Patient  2 Fractions  15 :  

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**